# Import Required Libraries

In [1]:
!pip install pdfplumber
!pip install langchain

  Using cached pdfplumber-0.11.0-py3-none-any.whl.metadata (39 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 177.8 kB/s eta 0:00:001m223.3 kB/s eta 0:00:01
Using cached pdfplumber-0.11.0-py3-none-any.whl (56 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 110.0 kB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached dataclasses_json-0.6.4-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
  Using cached marshmallow-3.21.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 230.1 kB/s eta 0:00:00 kB/s eta 0:00:01
   ━━━━━━━━━━━━━

In [3]:
pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.4 kB/s eta 0:00:00m eta 0:00:010:00:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 55.4 kB/s eta 0:00:000:00:010:00:01:02
Note: you may need to restart the kernel to use updated packages.


In [9]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/cv/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/cv/nltk_data...
[nltk_data] Downloading package omw-1.4 to /home/cv/nltk_data...


True

# Extract Text from PDF Files

## Import Libraries

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pdfplumber

## Process PDF

In [43]:
text = ''
# Open the PDF file
input_directory = "/home/cv/FYP_Chatbot/Dataset/Books/"
for filename in os.listdir(input_directory):
    if filename.endswith('.pdf'):
        with pdfplumber.open('/home/cv/FYP_Chatbot/Dataset/Books/04. Photovoltaic Solar Energy Autor Europe’s Energy.pdf') as pdf:
            # Loop through each page and extract text
            for page in pdf.pages:
                text = text + page.extract_text()

In [44]:
print(text[0],text[1],text[3])

P H T


## Splitting Parameters

In [45]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap= 50
)

## Slit the text

In [46]:
text = text_splitter.create_documents([text])

In [47]:
text[4]

Document(page_content='electricity. They can be connected in both series and More than 90 % of photovoltaic systems worldwide are\nparallel electrical arrangements to produce any required currently implemented as grid-connected systems. The\nvoltage and current combination. power conditioning unit also monitors the functioning\nof the system and the grid and switches off the system\nin case of faults.\nPhotovoltaic Installation\nAC outlets\nDC to AC\ninverter AC\ncircuit breaker boxes\ncharge DC\ncontroller\nbattery system\nPV array\nDC outlets\n6Photovoltaic market development\nThe current levels of dependence on fossil fuels, the\nneed of reducing the carbon emissions associated\n16 000.0\nwith energy use and the prospects of developing 16 000\na new and extremely innovative technology sector, make 12 000\n9 533.3\nphotovoltaics increasingly attractive. In the last years the 8 000 White Paper\n4 940.9\nphotovoltaic market expanded extensively, especially in 4 000 3 115.4 3 000.0\nGer

## Save the text

In [49]:
output_directory = '/home/cv/FYP_Chatbot/Dataset/TextFromBooks/Approach1/'
for index, chunk in enumerate(text, start = 1):
    with open(f'{output_directory}file1-chunk{index}.txt', 'w') as file:
        file.write(chunk.page_content)

## Approach 2

In [14]:
import os
import pdfplumber
from nltk.tokenize import word_tokenize

def tokenize_text(text):
    # Tokenize the text using NLTK
    tokens = word_tokenize(text)
    return tokens

def chunk_and_write_text(tokens, pdf_filename, output_directory, chunk_size=1000, overlap=50):
    num_chunks = (len(tokens) - chunk_size) // (chunk_size - overlap) + 1
    remainder = len(tokens) % (chunk_size - overlap)

    # Create the output directory if it does not exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Write each chunk to a separate file
    for i in range(num_chunks):
        start = i * (chunk_size - overlap)
        end = start + chunk_size
        chunk = tokens[start:end]

        # Construct output filename
        output_filename = f"{os.path.splitext(os.path.basename(pdf_filename))[0]}-part{i + 1}-overlap{overlap}.txt"
        output_path = os.path.join(output_directory, output_filename)

        # Write chunk to file
        with open(output_path, 'w') as file:
            file.write(' '.join(chunk))

def process_pdf_files(input_directory, output_directory):
    # Iterate over each PDF file in the directory
    for filename in os.listdir(input_directory):
        if filename.endswith(".pdf"):
            pdf_file = os.path.join(input_directory, filename)
            print(f"Processing: {pdf_file}")

            # Extract text from the PDF file
            with pdfplumber.open(pdf_file) as pdf:
                text = ''
                for page in pdf.pages:
                    text += page.extract_text()

            # Tokenize the text
            tokens = tokenize_text(text)

            # Chunk the tokens and write to separate files
            chunk_and_write_text(tokens, pdf_file, output_directory)

def main():
    input_directory = "/home/cv/FYP_Chatbot/Dataset/Books"
    output_directory = "/home/cv/FYP_Chatbot/Dataset/TextFromBooks"
    process_pdf_files(input_directory, output_directory)

if __name__ == "__main__":
    main()

Processing: /home/cv/FYP_Chatbot/Dataset/Books/03. Basic Research Needs for Solar Energy Utilization Autor Department of Energy.pdf
Processing: /home/cv/FYP_Chatbot/Dataset/Books/02. The Future of Solar Energy Autor MIT - Massachusetts Institute of Technology.pdf
Processing: /home/cv/FYP_Chatbot/Dataset/Books/06. Future of Solar Photovoltaic Autor IRENA – International Renewable Energy Agency.pdf
Processing: /home/cv/FYP_Chatbot/Dataset/Books/05. Solar Energy Principles and Possibilities Autor Christopher J Rhodes.pdf
Processing: /home/cv/FYP_Chatbot/Dataset/Books/01. Solar Energy Autor Klaus Jäger, Olindo Isabella, Arno H.M. Smets, René A.C.M.M. van Swaaij and Miro Zeman.pdf
Processing: /home/cv/FYP_Chatbot/Dataset/Books/04. Photovoltaic Solar Energy Autor Europe’s Energy.pdf
